In [27]:
# Import data analysis and visualisation packages.
import numpy as np
import pandas as pd
import stan as ps
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import arviz as az
import os
from patsy import dmatrix
from linearmodels.iv import IV2SLS

# Importing nest_asyncio is only necessary to run pystan in Jupyter Notebooks.
import nest_asyncio
nest_asyncio.apply()

#Specify the number of chains to the number of availible cpu's.
n_chains = os.cpu_count()
n_samples = 1000
#Convert to int so Stan will not crash below.
n_warmup = int(n_samples/2)
#Specify step size.
stepS = .8

In [28]:
def read_data(file): 
    return pd.read_stata("https://github.com/scunning1975/mixtape/raw/master/"  + file)

card = read_data("card.dta")
card = card[['lwage', 'exper','black', 'south', 'married', 'smsa','educ', 'nearc4']]
cardNNA = card.dropna(axis=0)

In [33]:
IVMod = '''
data{
int<lower = 1> N;
int K;
int K1;
int K2;
vector[K] y[N];
matrix[N, K1] X1;
matrix[N, K2] X2;
}
parameters{
// Regression parameters
vector[K1] beta1;  
vector[K2] beta2;  
vector<lower =0>[K] sigma;
// Correlation matrix
corr_matrix[K] rho;

}
transformed parameters{
// generate mu for each linear model
vector[N] mu1 = X1 * beta1;
vector[N] mu2 = X2 * beta2;
// generate a tempary matrix varaible
matrix[N, K] mu_temp = append_col(mu1, mu2);
// Convert that matrix into vectors for use by mulit-normal dist
// below in the likelihood
row_vector[K] mu[N];
for (n in 1:N)
    mu[n] = mu_temp[n];
}

model{
// Priors
sigma ~ exponential(1);

// Covariance matrix
matrix[K,K] Sigma;
// Uniform prior for correlation parameters
rho ~ lkj_corr(1);
Sigma = quad_form_diag(rho,sigma);

// linear model priors
beta1 ~ std_normal();
beta2 ~ std_normal();

// Likelihood
y ~ multi_normal(mu, Sigma);
}
'''

In [34]:
x1 = np.asarray(dmatrix('~1 + exper + black + south + married + smsa', data = cardNNA))
x2 = np.asarray(dmatrix(' ~ nearc4', data = cardNNA))
y = np.asarray(cardNNA[['lwage', 'educ']])

In [35]:
d ={'N': len(cardNNA),
    'y': y,
    'K': np.shape(y)[1],
    'K1': np.shape(x1)[1],
    'K2': np.shape(x2)[1],
    'X1': x1,
    'X2': x2
}

In [36]:
sm = ps.build(IVMod, data = d)

Building: 0.2s
Building: 0.3s
Building: 0.4s
Building: 0.5s
Building: 0.6s
Building: 0.7s
Building: 0.8s
Building: 0.9s
Building: 1.0s
Building: 1.1s
Building: 1.2s
Building: 1.3s
Building: 1.4s
Building: 1.5s
Building: 1.6s
Building: 1.7s
Building: 1.8s
Building: 1.9s
Building: 2.0s
Building: 2.1s
Building: 2.2s
Building: 2.3s
Building: 2.4s
Building: 2.5s
Building: 2.6s
Building: 2.7s
Building: 2.8s
Building: 2.9s
Building: 3.0s
Building: 3.1s
Building: 3.2s
Building: 3.3s
Building: 3.4s
Building: 3.6s
Building: 3.7s
Building: 3.8s
Building: 3.9s
Building: 4.0s
Building: 4.1s
Building: 4.2s
Building: 4.3s
Building: 4.4s
Building: 4.5s
Building: 4.6s
Building: 4.7s
Building: 4.8s
Building: 4.9s
Building: 5.0s
Building: 5.1s
Building: 5.2s
Building: 5.3s
Building: 5.4s
Building: 5.5s
Building: 5.6s
Building: 5.7s
Building: 5.8s
Building: 5.9s
Building: 6.0s
Building: 6.1s
Building: 6.2s
Building: 6.3s
Building: 6.4s
Building: 6.5s
Building: 6.6s
Building: 6.7s
Building: 6.8s
Building: 

In file included from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/functor.hpp:12,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/rev/fun.hpp:198,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/rev.hpp:10,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math.hpp:19,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /home/harrison/.cache/httpstan/4.6.1/models/ps5ecrbn/model_ps5ecrbn.cpp:2:
/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/functor/hcubature.hpp: In function 'void stan::math::internal::combination(std::vector<int>&, const int&, const int&, const int

Building: 9.6s
Building: 9.7s
Building: 9.8s
Building: 9.9s
Building: 10.0s
Building: 10.1s


In file included from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/boost/multi_array/multi_array_ref.hpp:32,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/boost/multi_array.hpp:34,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/boost/numeric/odeint/algebra/multi_array_algebra.hpp:22,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/boost/numeric/odeint.hpp:63,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/functor/ode_rk45.hpp:9,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/functor/integrate_ode_rk45.hpp:6,
                 from /home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/inc

Building: 10.2s
Building: 10.3s
Building: 10.4s
Building: 10.5s
Building: 10.6s
Building: 10.7s
Building: 10.8s
Building: 10.9s
Building: 11.1s
Building: 11.2s
Building: 11.3s
Building: 11.4s
Building: 11.5s
Building: 11.6s
Building: 11.7s
Building: 11.8s
Building: 11.9s
Building: 12.0s
Building: 12.1s
Building: 12.2s
Building: 12.3s
Building: 12.4s
Building: 12.5s
Building: 12.6s
Building: 12.7s


...

/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/err/elementwise_check.hpp: In instantiation of 'void stan::math::elementwise_check(const F&, const char*, const char*, const T&, const char*, const Indexings& ...) [with F = check_nonnegative<Eigen::Array<double, -1, 1> >(const char*, const char*, const Eigen::Array<double, -1, 1>&)::<lambda(double)>; T = Eigen::Array<double, -1, 1>; Indexings = {}; stan::require_eigen_t<S>* <anonymous> = 0; std::enable_if_t<((Eigen::internal::traits<_Rhs>::Flags & Eigen::LinearAccessBit) || T::IsVectorAtCompileTime)>* <anonymous> = 0]':
/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/err/check_nonnegative.hpp:26:20:   required from 'void stan::math::check_nonnegative(const char*, const char*, const T_y&) [with T_y = Eigen::Array<double, -1, 1>]'
/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/pr

Building: 22.7s
Building: 22.8s


/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/err/elementwise_check.hpp: In instantiation of 'void stan::math::elementwise_check(const F&, const char*, const char*, const T&, const char*, const Indexings& ...) [with F = check_not_nan<Eigen::Matrix<double, -1, -1> >(const char*, const char*, const Eigen::Matrix<double, -1, -1>&)::<lambda(double)>; T = Eigen::Matrix<double, -1, -1>; Indexings = {}; stan::require_eigen_t<S>* <anonymous> = 0; std::enable_if_t<(((!(Eigen::internal::traits<_Rhs>::Flags & Eigen::LinearAccessBit)) && (! T::IsVectorAtCompileTime)) && (!(Eigen::internal::traits<_Rhs>::Flags & Eigen::RowMajorBit)))>* <anonymous> = 0]':
/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/httpstan/include/stan/math/prim/err/check_not_nan.hpp:28:20:   required from 'void stan::math::check_not_nan(const char*, const char*, const T_y&) [with T_y = Eigen::Matrix<double, -1, -1>]'
/home/harrison/anaconda3/envs/pys

Building: 22.9s
Building: 23.0s
Building: 23.1s
Building: 23.2s
Building: 23.3s
Building: 23.4s
Building: 23.5s
Building: 23.6s
Building: 23.7s
Building: 23.8s
Building: 23.9s
Building: 24.0s
Building: 24.1s
Building: 24.2s
Building: 24.3s
Building: 24.4s
Building: 24.5s
Building: 24.6s
Building: 24.7s
Building: 24.8s
Building: 24.9s
Building: 25.0s
Building: 25.1s
Building: 25.2s
Building: 25.3s
Building: 25.4s
Building: 25.5s
Building: 25.6s
Building: 25.7s
Building: 25.9s
Building: 26.0s
Building: 26.1s
Building: 26.2s
Building: 26.3s
Building: 26.4s
Building: 26.5s
Building: 26.6s
Building: 26.7s
Building: 26.8s
Building: 26.9s
Building: 27.0s
Building: 27.1s
Building: 27.2s
Building: 27.3s
Building: 27.4s
Building: 27.5s
Building: 27.6s
Building: 27.7s
Building: 27.8s
Building: 27.9s
Building: 28.0s
Building: 28.1s
Building: 28.2s
Building: 28.3s
Building: 28.4s
Building: 28.5s
Building: 28.6s
Building: 28.7s
Building: 28.8s
Building: 28.9s
Building: 29.0s
Building: 29.1s
Building

Building: 45.8s, done.
Messages from stanc:
Warning in '/tmp/httpstan_bvc0om1o/model_ps5ecrbn.stan', line 7, column 0: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_bvc0om1o/model_ps5ecrbn.stan', line 28, column 0: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_bvc0om1o/model_ps5ecrbn.stan', line 40, column 0: It
    is suggested to reparameterize your model to replace lkj_corr with
    lkj_corr_cholesky, the Cholesky factor variant. lkj_corr tends to run
    slower, consume more memory, and has higher risk of numerical errors.


In [37]:
fit = sm.sample(num_chains = n_chains , num_samples = n_samples, num_warmup = n_warmup, stepsize = stepS)

Sampling:   0%
Sampling:   0% (1/12000)
Sampling:   0% (2/12000)
Sampling:   0% (3/12000)
Sampling:   0% (4/12000)
Sampling:   0% (5/12000)
Sampling:   0% (6/12000)
Sampling:   0% (7/12000)
Sampling:   0% (8/12000)
Sampling:   1% (107/12000)
Sampling:   2% (206/12000)
Sampling:   3% (305/12000)
Sampling:   3% (404/12000)
Sampling:   4% (503/12000)
Sampling:   5% (602/12000)
Sampling:   6% (701/12000)
Sampling:   7% (800/12000)
Sampling:   8% (900/12000)
Sampling:   8% (1000/12000)
Sampling:   9% (1100/12000)
Sampling:  10% (1200/12000)
Sampling:  11% (1300/12000)
Sampling:  12% (1400/12000)
Sampling:  12% (1500/12000)
Sampling:  13% (1600/12000)
Sampling:  14% (1700/12000)
Sampling:  15% (1800/12000)
Sampling:  16% (1900/12000)
Sampling:  17% (2000/12000)
Sampling:  18% (2100/12000)
Sampling:  18% (2200/12000)
Sampling:  19% (2300/12000)
Sampling:  20% (2400/12000)
Sampling:  21% (2500/12000)
Sampling:  22% (2600/12000)
Sampling:  23% (2701/12000)
Sampling:  23% (2802/12000)
Sampling: 

  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan__bchfh3t/model_ps5ecrbn.stan', line 40, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan__bchfh3t/model_ps5ecrbn.stan', line 40, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because 

  Gradient evaluation took 0.002131 seconds
  1000 transitions using 10 leapfrog steps per transition would take 21.31 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan__bchfh3t/model_ps5ecrbn.stan', line 40, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan__bchfh3t/model_ps5ecrbn.stan', line 40, column 0 to column 18)
  If this warning occurs sporadically, such as for h

  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan__bchfh3t/model_ps5ecrbn.stan', line 40, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan__bchfh3t/model_ps5ecrbn.stan', line 40, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because 

  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan__bchfh3t/model_ps5ecrbn.stan', line 40, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan__bchfh3t/model_ps5ecrbn.stan', line 40, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-condi

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan__bchfh3t/model_ps5ecrbn.stan', line 40, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan__bchfh3t/model_ps5ecrbn.stan', line 40, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, th

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan__bchfh3t/model_ps5ecrbn.stan', line 40, column 0 to column 18)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: lkj_corr_lpdf: Correlation matrix is not positive definite. (in '/tmp/httpstan__bchfh3t/model_ps5ecrbn.stan', line 40,

In [38]:
az.summary(fit, var_names = ['beta1', 'beta2', 'rho'])

/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/arviz/stats/diagnostics.py:592: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta1[0],5.999,0.028,5.948,6.052,0.000,0.000,4920.0,5385.0,1.0
beta1[1],0.034,0.002,0.029,0.038,0.000,0.000,5600.0,5870.0,1.0
beta1[2],-0.168,0.018,-0.199,-0.133,0.000,0.000,7488.0,6217.0,1.0
beta1[3],-0.139,0.015,-0.165,-0.109,0.000,0.000,6877.0,5819.0,1.0
beta1[4],-0.036,0.003,-0.042,-0.029,0.000,0.000,10518.0,6411.0,1.0
beta1[5],0.194,0.016,0.164,0.223,0.000,0.000,6254.0,5750.0,1.0
beta2[0],12.751,0.079,12.609,12.902,0.001,0.001,4659.0,5130.0,1.0
beta2[1],0.711,0.093,0.533,0.886,0.001,0.001,4839.0,5364.0,1.0
"rho[0, 0]",1.000,0.000,1.000,1.000,0.000,0.000,8000.0,8000.0,NaN
"rho[0, 1]",0.445,0.019,0.411,0.482,0.000,0.000,4979.0,5393.0,1.0


In [40]:
iv_reg = IV2SLS.from_formula("lwage ~ 1 + exper + black + south + married + smsa + [educ ~ nearc4 ]", card).fit()
iv_reg.summary

/home/harrison/anaconda3/envs/pystan_env/lib/python3.11/site-packages/linearmodels/iv/model.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                  lwage   R-squared:                      0.2513
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2498
No. Observations:                3003   F-statistic:                    892.71
Date:                Sun, Jul 16 2023   P-value (F-stat)                0.0000
Time:                        14:49:17   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      4.1625     0.8349     4.9857     0.0000      2.5262      5.7988
exper          0.0556     0.0199     2.7980     0.0051      0.0166      0.0945
black         -0.1157     0.0496    -2.3343     0.0196     -0.2128     -0.0186
south         -0.1132     0.0229    -4.9314     0.0000     -0.1581     -0.0682
married       -0.0320     0.0051    -6.3037     0.0000     -0.0419     -0.0220
smsa           0.1477     0.0303     4.8721     0.0000      0.0883      0.2071
educ           0.1242     0.0492     2.5258     0.0115      0.0278      0.2205
==============================================================================

Endogenous: educ
Instruments: nearc4
Robust Covariance (Heteroskedastic)
Debiased: False
"""